## Example: Minimum TE

### Import

In [32]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

### Load 

In [33]:
prices = pd.read_excel("20210831 - SMI.xlsx", sheet_name="RI M", index_col=0, parse_dates=True)
marketCap = pd.read_excel("20210831 - SMI.xlsx", sheet_name="MV M", index_col=0, parse_dates=True)
returns = prices.pct_change().dropna()

In [35]:
weightsMC = marketCap.shift(1).apply(lambda x: x/x.sum(), axis=1)
#weightsMC.iloc[-1].plot(kind="bar")

### One period optimization

Covariance Matrix

In [36]:
sigma = returns.iloc[-60:].cov()

Optimisation: min Tracking Error

In [37]:
#Tracking Error: measure of risk between the return fluctuations of an investment portfolio and
#the return fluctuations of a chosen benchmark

#The return fluctuations are primarily measured by standard deviations

#Low errors indicate that the performance of the portfolio is close to the performance of the benchmark

In [46]:
Res=[]
Weight_last=[] 
TE_last=[]

wMClastDate = weightsMC.iloc[-1]
numberOfAssets = wMClastDate.shape[0]
x0 = np.ones(numberOfAssets)/numberOfAssets

objectiveFunction = lambda x: ((x-wMClastDate).T @ sigma @ (x-wMClastDate))**0.5*(12**0.5)
#objectiveFunction = lambda x: 1

budgetConstraint = LinearConstraint(np.ones(numberOfAssets), 1, 1)
positivityConstraint = LinearConstraint(np.eye(numberOfAssets), 0, 1)
constraints = [budgetConstraint, positivityConstraint]

history = []
def callback(x):
    fobj = objectiveFunction(x)
    history.append(fobj)

results = minimize(objectiveFunction, x0, constraints=constraints)
#results = minimize(objectiveFunction, x0, constraints=constraints, callback = callback)

wOptimized = pd.Series(results.x, index=wMClastDate.index)

Res.append(results)
Weight_last.append(wOptimized)

TE = results.fun

TE_last.append(TE)

#print("TE", history)
print(f"Annualized TE: {TE:.2%}")
#print("Results", Res[0])

Annualized TE: 0.00%


In [47]:
results

     fun: 3.6659395964571886e-06
     jac: array([-0.05660306, -0.02742653, -0.02750202, -0.08188787, -0.08011116,
       -0.01817269, -0.12982364, -0.12763245, -0.06301732, -0.0470976 ,
       -0.00923154, -0.16006118, -0.15722216, -0.06074023, -0.02816684,
       -0.17319729, -0.13034239, -0.15932041,  0.00571322, -0.00969064])
 message: 'Optimization terminated successfully'
    nfev: 1046
     nit: 46
    njev: 46
  status: 0
 success: True
       x: array([0.01571949, 0.01997444, 0.02955232, 0.03898001, 0.14649872,
       0.17426718, 0.01709571, 0.02315084, 0.04815874, 0.00619194,
       0.03717099, 0.010443  , 0.02926735, 0.01889078, 0.22880557,
       0.01840697, 0.04287746, 0.03916214, 0.03204174, 0.02334462])

In [48]:
Weight_last

[SGS 'N'                   0.015719
 SWISSCOM 'R'              0.019974
 GIVAUDAN 'N'              0.029552
 ZURICH INSURANCE GROUP    0.038980
 NOVARTIS 'R'              0.146499
 ROCHE HOLDING             0.174267
 CREDIT SUISSE GROUP       0.017096
 HOLCIM                    0.023151
 ABB LTD N                 0.048159
 THE SWATCH GROUP          0.006192
 LONZA GROUP               0.037171
 SWISS LIFE HOLDING        0.010443
 PARTNERS GROUP HOLDING    0.029267
 GEBERIT 'R'               0.018891
 NESTLE 'N'                0.228806
 SWISS RE                  0.018407
 RICHEMONT N               0.042877
 UBS GROUP                 0.039162
 SIKA                      0.032042
 ALCON (SWX) ORD SHS       0.023345
 dtype: float64]

Graph

In [42]:
fig = go.Figure()
fig.add_trace(go.Bar(x=emissions.columns, y=wMClastDate, name="Market cap weights"))
fig.add_trace(go.Bar(x=emissions.columns, y=wOptimized, name="Optimized weights"))
fig.show()

Lou-Salomé Vallée: lou-salome.vallee@unil.ch 

2022-2023